In [1]:
import pandas as pd

C:\Users\ektam\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ektam\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ektam\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [116]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [117]:
train.head()

,serial_number,time_stamp,Sector 1,Sector 2,Sector 3,Sector 4
0,1,2016-03-21,1125.36,458.49,2048.98,1275.00
1,2,2016-03-18,1120.10,458.49,2048.98,1242.46
2,3,2016-03-17,1120.10,458.49,2048.98,1242.46
3,4,2016-03-16,1117.84,458.49,2048.98,1228.52
4,5,2016-03-15,1114.08,458.49,2048.98,1205.28


In [118]:
train.set_index('serial_number',inplace=True)
test.set_index('serial_number',inplace=True)


In [119]:
train['year'] = pd.DatetimeIndex(train['time_stamp']).year
train['month'] = pd.DatetimeIndex(train['time_stamp']).month
train['day'] = pd.DatetimeIndex(train['time_stamp']).day
test['year'] = pd.DatetimeIndex(test['time_stamp']).year
test['month'] = pd.DatetimeIndex(test['time_stamp']).month
test['day'] = pd.DatetimeIndex(test['time_stamp']).day


In [120]:
train=train.drop('time_stamp',axis=1)
test=test.drop('time_stamp',axis=1)

In [121]:
X=train.drop('Sector 3',axis=1)

In [122]:
y=train['Sector 3']

In [143]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [132]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [146]:
from sklearn.model_selection import GridSearchCV   #Perforing grid search

parameters_for_testing = {
    'colsample_bytree':[0.4,0.6,0.8],
    'gamma':[0,0.03,0.1,0.3],
    'min_child_weight':[1.5,6,10],
   'learning_rate':[0.1,0.07],
   'max_depth':[3,5],
   'n_estimators':[10000],
    'reg_alpha':[1e-5, 1e-2,  0.75],
    'reg_lambda':[1e-5, 1e-2, 0.45],
    'subsample':[0.6,0.95]  
}

                    
xgb_model = xgb.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
     min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')
gsearch1.fit(X_train,y_train)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

C:\Users\ektam\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   18.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   19.1s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   48.4s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  3.7min
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:  4.3min
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:  5.0min
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:  5.7min
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:  6.3min
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:  7.4min
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  8.4min
[Parallel(n_jobs=6)]: Do

KeyboardInterrupt: 

In [136]:
y_pred = xgb_model.predict(X_test)


In [140]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 625.6838030338542
Mean Squared Error: 404406.80670081277
Root Mean Squared Error: 635.9298756158676


In [34]:
print('R2 score:', metrics.r2_score(y_test, y_pred))

R2 score: 0.9988420940704776


In [94]:
y_pred = regressor.predict(test)

In [95]:
test['Sector 3']=y_pred

In [96]:
result=test.copy()

In [97]:
result=result.drop(['Sector 1','Sector 2','Sector 4','year','month','day'],axis=1)

In [98]:
result.head()

,Sector 3
serial_number,
1,2047.689485
2,2047.645495
3,2047.645495
4,2047.645495
5,2047.645495


In [99]:
result.to_csv('result.csv')